## KOSPI 200
- KOSPI 200 종목들의 가격과 거래량을 기준으로 종목들을 clustering 해본다.
- 높은 차원에서 clustering한 결과를 시각화하거나 일정기간 중의 변화를 하나의 관찰값으로 취급하여 clustering을 해볼 수 있다.

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

## 자료 준비
### finance-datareader

In [2]:
stocks = pd.read_excel('kospi200.xlsx', dtype = 'object')
stocks.index = stocks.종목코드
stocks.drop(['종목코드'], axis = 1, inplace = True)
stocks.iloc[:,3] = stocks.iloc[:,3].astype(float)
stocks.iloc[:,[1,2,4]] = stocks.iloc[:,[1,2,4]].astype(int)

In [3]:
stocks

,종목명,종가,대비,등락률,상장시가총액
종목코드,,,,,
005930,삼성전자,59200,300,0.51,353411127
373220,LG에너지솔루션,531000,-32000,-5.68,124254000
207940,삼성바이오로직스,827000,-4000,-0.48,58860898
000660,SK하이닉스,78800,-100,-0.13,57366586
006400,삼성SDI,693000,-15000,-2.12,47653819
...,...,...,...,...,...
284740,쿠쿠홈시스,32050,-450,-1.38,719116
294870,HDC현대산업개발,10550,-350,-3.21,695322
057050,현대홈쇼핑,53000,-100,-0.19,636000


### Kospi 200 목록
- finance-datareader을 사용하면 아래 과정은 건너뛴다.
- pandas_datareader를 사용하려면 종목코드(ticker)를 별도로 만들어야 하므로 한국거래소 정보데이터시스템에서 편입종목과 산업분류현황 파일을 내려받는다.
- 편입종목은 KOSPI200 기업, 업종분류현황은 거래소의 산업분류를 포함하고 있다.
- Encoding이 utf-8이 아니라면 cp949나 euc-kr를 사용해 본다.
- 거래소는 KRX 지수 산업 분류를 사용하고 있으며 국제기준인 GICS를 사용하려면 한국판인 WICS를 이용한다.
<br>

**KRX 대분류 24**
- 서비스업, 기타금융, 유통업, 섬유의복, 운수창고업, 음식료품, 증권, 보험, 전기전자, 건설업, 화학, 철강금속, 광업, 기계, 의약품, 비금속광물, 운수장비, 통신업, 기타제조업, 전기가스업, 종이목재, 은행, 의료정밀, 농업, 임업 및 어업
<br>

**WICS 대분류 10**
<br>

- IT, 건강관리, 소재, 커뮤니케이션서비스, 경기관련소비재, 산업재, 금융, 에너지, 유틸리티, 필수소비재
<br>

**WICS 중분류 28**
<br>

- 에너지, 소재, 자본재, 상업서비스와공급품, 운송, 자동차와부품, 내구소비재와의류, 호텔,레스토랑,레저 등, 소매(유통), 교육서비스, 식품과기본식료품소매, 식품,음료,담배, 가정용품과개인용품, 건강관리장비와서비스, 제약과생물공학, 은행, 증권, 다각화된금융, 보험, 부동산, 소프트웨어와서비스, 기술하드웨어와장비, 반도체와반도체장비, 전자와 전기제품, 디스플레이, 전기통신서비스, 미디어와엔터테인먼트, 유틸리티
<br>

- 종목코드를 datareader에 입력할때는 .KS를 붙여 사용한다.